### Output Parsers for LLM Input / Output with LangChain 

In [2]:
# !pip install langchain==0.3.11
# !pip install langchain-openai==0.2.12
# !pip install langchain-community==0.3.11

In [3]:
from getpass import getpass 

OPENAI_KEY = getpass('Please enter your OpenAI API key here: ')

### Setup system environment variable 

In [7]:
import os 

os.environ['OPENAI_API_KEY'] = OPENAI_KEY 

### Chat Models and LLMs 

In [8]:
from langchain_openai import ChatOpenAI 

chatgpt = ChatOpenAI(model_name="gpt-40-mini", temperature=0)

### Pydantic Output Parser 

In [10]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field 

# Define your desired data structure - like a python data class 
class QueryResponse(BaseModel):
    description: str = Field(description="A brief description of the topic asked by the user")
    pros: str = Field(description="3 bullet points showing the pros of the topic asked by the user")
    cons: str = Field(description="3 bullet points showing the cons of the topic asked by the user")
    conclusion: str = Field("One line conclusion of the topic asked by the user")
    
# Set up a parser + inject instructions into the prompt template 
parser = PydanticOutputParser(pydantic_object=QueryResponse)
parser 

PydanticOutputParser(pydantic_object=<class '__main__.QueryResponse'>)

In [11]:
# langchain pre-generated output response formatting instructions 
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"description": {"description": "A brief description of the topic asked by the user", "title": "Description", "type": "string"}, "pros": {"description": "3 bullet points showing the pros of the topic asked by the user", "title": "Pros", "type": "string"}, "cons": {"description": "3 bullet points showing the cons of the topic asked by the user", "title": "Cons", "type": "string"}, "conclusion": {"default": "One line conclusion of the topic asked by the user", "title": "Conclusion", "type": "string"}}, "required": ["description", 

In [ ]:
prompt_txt = """
            Answer the user query and generate the response based on the following formatting instructions 
            
            Format Instructions: 
            {format_instructions}
            
            Query:
            {query} 
            """
prompt = PromptTemplate(
    template=prompt_txt, 
    input_variables=["query"], 
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
prompt 